In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/gdrive')

KeyboardInterrupt: ignored

In [4]:
import pickle
with open('/gdrive/My Drive/Stock Market Research/nifty/standardized_minute/lstm_X20_y5_data.pickle','rb') as fh:
  X_y=pickle.load(fh)

In [5]:
X_y.columns

Index(['X', 'y'], dtype='object')

In [6]:
X_y.shape

(482625, 2)

In [7]:
np.array(X_y['X'].values.tolist()).shape

(482625, 20, 16)

In [8]:
351*30

10530

In [9]:
482625-351*30

472095

In [10]:
train_data=np.array(X_y['X'][:472095].values.tolist())
test_data=np.array(X_y['X'][472095:].values.tolist())
train_target=np.array(X_y['y'][:472095].values.tolist())
test_target=np.array(X_y['y'][472095:].values.tolist())

del X_y

In [11]:
train_data.shape

(472095, 20, 16)

In [12]:
train_target.shape

(472095, 5)

In [17]:
import os
try:
    device_name = os.environ['COLAB_TPU_ADDR']
    TPU_ADDRESS = 'grpc://' + device_name
    print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
    print('TPU not found')


Found TPU at: grpc://10.83.132.250:8470


In [1]:
import time
start_time = time.time()

model=tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(20,16)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300,activation='relu'))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(20,activation='relu'))
model.add(tf.keras.layers.Dense(5,activation='relu'))

early_stopping_callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=True,mode='min',patience=20,verbose=1)

model.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer=tf.keras.optimizers.Adam(0.002),metrics=[tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError()])

model.fit(train_data,train_target,epochs=200,validation_split=0.2,callbacks=[early_stopping_callback],batch_size=15000)

end_time = time.time()

print(end_time-start_time)

NameError: ignored

In [ ]:
model.evaluate(test_data,test_target)

In [ ]:
351*30

In [ ]:
import os
os.chdir("/gdrive/My Drive/Stock Market Research/nifty/standardized_minute/")

with open('output_scaler.pickle','rb') as fh:
  output_scaler=pickle.load(fh)

for i, j  in zip(model.predict(test_data),test_target):
  print("{} {}".format(output_scaler.inverse_transform(np.array(i).reshape(-1,1)).reshape(-1,),output_scaler.inverse_transform(np.array(j).reshape(-1,1)).reshape(-1,)))

In [ ]:
import time
start_time = time.time()

model=tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(20,16)))
model.add(tf.keras.layers.LSTM(200,return_sequences=True,recurrent_dropout=0.2,kernel_initializer='glorot_uniform',bias_initializer='glorot_uniform',activity_regularizer=tf.keras.regularizers.L2()))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(5,activation='relu'))

early_stopping_callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=True,mode='min',patience=20,verbose=1)

model.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer=tf.keras.optimizers.Adam(0.001),metrics=[tf.keras.metrics.RootMeanSquaredError(),tf.keras.metrics.MeanAbsoluteError()])

model.fit(train_data,train_target,epochs=500,validation_split=0.2,callbacks=[early_stopping_callback],batch_size=10000)

end_time = time.time()

print(end_time-start_time)